In [13]:
#import the necessary module
import math

#create a class called Feature
class Feature(object): 
    #create necessary methods within the class (Feature) to use for the assignment 
    
    # The __init__ method is a special function that is run when an object is created
    def __init__(self, featureDict):
        self.geometry = featureDict["geometry"]
        self.properties = featureDict["properties"]
        self.airport_list = []
        
        # Minimum bounding rectangle is stored as 
        # [[upperleft x, upperleft y], [lowerright x, lowerright y]]
        self.mbr = [[float('inf'),-float('inf')],[-float('inf'),float('inf')]]
        if self.geometry["type"] == "Polygon":
            self.mbr = self.getPolygonMBR(self.geometry["coordinates"])
        elif self.geometry["type"] == "MultiPolygon":
            self.mbr = self.getMultiPolygonMBR(self.geometry["coordinates"])
        else:
            print("Not a valid geometry")
    
    #method to see if a point is within the MBR
    def isPointInMBR(self, point):
        """
        Takes point as (x,y) pair
        """
        return point[0] > self.mbr[0][0] and point[0] < self.mbr[1][0] and point[1] < self.mbr[0][1] and point[1] > self.mbr[1][1] 
    
    #method to define the MBR of the polygon
    def getPolygonMBR(self, polygon_coords):
        mbr = [[float('inf'),-float('inf')],[-float('inf'),float('inf')]]
        for point in polygon_coords[0]: # Only check the bounds of the first (outer) ring
            if point[0] < mbr[0][0]:
               mbr[0][0] = point[0]
            if point[0] > mbr[1][0]:
               mbr[1][0] = point[0]
            if point[1] > mbr[0][1]:
               mbr[0][1] = point[1]
            if point[1] < mbr[1][1]:
               mbr[1][1] = point[1]
        return mbr
    
    #method to get the MBR for multipolygons
    def getMultiPolygonMBR(self, multipolygon_coords):
        mbr = [[float('inf'),-float('inf')],[-float('inf'),float('inf')]]
        for polygon in multipolygon_coords:
            tmbr = self.getPolygonMBR(polygon)
            # Check if any coordinate in the polygon's MBR extends the existing MBR
            if tmbr[0][0] <  mbr[0][0]:
                mbr[0][0] = tmbr[0][0]
            if tmbr[1][0] >  mbr[1][0]:
                mbr[1][0] = tmbr[1][0]
            if tmbr[0][1] >  mbr[0][1]:
                mbr[0][1] = tmbr[0][1]
            if tmbr[1][1] <  mbr[1][1]:
                mbr[1][1] = tmbr[1][1]
        return mbr

Eventually I will place these in a text document and simply just call the methods, instead of having them written out in my code.  

In [36]:
#import the necessary module
import json

#read in the countries file
f = "countries.geojson"
with open(f, 'r') as infile:
    g = json.load(infile)

#move features from the JSON file to the newly created class: Feature
features = []
for f in g["features"]:
    features.append(Feature(f))

## For each Feature add a new attribute that lists the airports contained in its MBR

In [39]:
#import points module used in E5 to read in cvs file
import points1

#bring in and read the airport data
data = 'airports.csv'
newlines = points1.read_csv(data)
header = newlines[0] #create this to use for indexing in the next cell

In [45]:
#index the appropriate columns to reference for the airport list
lat = header.index("latitude")
lon = header.index("longitude")
idx = header.index("airport_id")
airport_list = []

#loop through all features and all airports for each feature in the newlines dataset
#add airports that are within the MBR to the list airport_list using the variables above
for country in features:
    for airport in newlines:
        if country.isPointInMBR((airport[lon], airport[lat])):
            country.airport_list.append(airport[idx])